In [1]:
# return named tuple (x=?, y=?) of points
function parse_line(line)
    if line[1] == 'x'
        x, y1, y2 = parse.(Int, match(r"x=(.*), y=(.*)\.\.(.*)", line).captures)
        [(x,y) for y in UnitRange(y1, y2)]
    else
        y, x1, x2 = parse.(Int, match(r"y=(.*), x=(.*)\.\.(.*)", line).captures)
        [(x,y) for x in UnitRange(x1, x2)]
    end
end

# parse file and return clay points
function parse_file(filename)
    points = Tuple{Int64,Int64}[]
    for line in readlines(filename)
        append!(points, parse_line(line))
    end
    points
end

points = parse_file("input17_sample.txt")

38-element Array{Tuple{Int64,Int64},1}:
 (495, 2) 
 (495, 3) 
 (495, 4) 
 (495, 5) 
 (495, 6) 
 (495, 7) 
 (495, 7) 
 (496, 7) 
 (497, 7) 
 (498, 7) 
 (499, 7) 
 (500, 7) 
 (501, 7) 
 ⋮        
 (498, 13)
 (504, 10)
 (504, 11)
 (504, 12)
 (504, 13)
 (498, 13)
 (499, 13)
 (500, 13)
 (501, 13)
 (502, 13)
 (503, 13)
 (504, 13)

In [2]:
# symbols are :sand, :clay, and :water
function create_grid(points)
    xmax = maximum(p[1] for p in points)
    ymax = maximum(p[2] for p in points)
    grid = fill(:sand, (ymax, xmax))
    for (x,y) in points
        grid[y, x] = :clay
    end
    grid
end
grid = create_grid(points)

13×506 Array{Symbol,2}:
 :sand  :sand  :sand  :sand  :sand  …  :sand  :sand  :sand  :sand  :clay
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :sand  :sand  :clay
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :sand  :sand  :sand
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :sand  :sand  :sand
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :sand  :sand  :sand
 :sand  :sand  :sand  :sand  :sand  …  :sand  :sand  :sand  :sand  :sand
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :sand  :sand  :sand
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :sand  :sand  :sand
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :sand  :sand  :sand
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :clay  :sand  :sand
 :sand  :sand  :sand  :sand  :sand  …  :sand  :sand  :clay  :sand  :sand
 :sand  :sand  :sand  :sand  :sand     :sand  :sand  :clay  :sand  :sand
 :sand  :sand  :sand  :sand  :sand     :clay  :clay  :clay  :sand  :sand

In [3]:
function visualize(grid; left=1, right=size(grid,2), top=1, bottom=size(grid,1))
    for y in top:bottom
        for x in left:right
            grid[y,x] == :water && print("~")
            grid[y,x] == :clay && print("#")
            grid[y,x] == :sand && print(".")
        end
        println()
    end
end
visualize(grid, left = 494)

............#
.#..#.......#
.#..#..#.....
.#..#..#.....
.#.....#.....
.#.....#.....
.#######.....
.............
.............
....#.....#..
....#.....#..
....#.....#..
....#######..


In [4]:
# walk down until reaches clay
function reach_bottom!(x, y, grid)
    while grid[y, x] != :clay
        grid[y, x] = :water
        y += 1
        y > size(grid, 1) && return nothing
    end
    (x, y-1)  # last sand
end
@show reach_bottom!(500, 1, grid)
visualize(grid, left = 494)

reach_bottom!(500, 1, grid) = (500, 6)
......~.....#
.#..#.~.....#
.#..#.~#.....
.#..#.~#.....
.#....~#.....
.#....~#.....
.#######.....
.............
.............
....#.....#..
....#.....#..
....#.....#..
....#######..


In [5]:
function left_edge(x, y, grid)
    while x > 1
        x -= 1
        grid[y, x] == :clay && return (x=x, y=y)
    end
    nothing
end

function right_edge(x, y, grid)
    while x < size(grid, 2)
        x += 1
        grid[y, x] == :clay && return (x=x, y=y)
    end
    nothing
end

@show left_edge(500, 12, grid)
@show right_edge(500, 12, grid)

left_edge(500, 12, grid) = (x = 498, y = 12)
right_edge(500, 12, grid) = (x = 504, y = 12)


(x = 504, y = 12)

In [6]:
# if bounded in both sides then return the volume
function bounded(x, y, grid)
    l, r = left_edge(x, y, grid), right_edge(x, y, grid)
    if l != nothing && r != nothing
        leak = any(grid[y+1, l.x+1:r.x-1] .== :sand) 
    else
        leak = true
    end
    l, r, leak
end

bounded (generic function with 1 method)

In [7]:
# find a drop location (x coordinate) that (x, y+1) is not clay
function find_drop!(x, y, grid, direction, level)
    grid[y,x] == :clay && return nothing
    if direction == :right
        r = x+1:size(grid,2)
    else
        r = x-1:-1:1
    end
    println(level, ": r = ", r, " y=", y)

#     r = x+1:size(grid,2)
#     r = x-1:-1:1
    for xp in r
        grid[y, xp] != :clay && (grid[y, xp] = :water)
        grid[y+1, xp] == :sand && return xp
    end
    nothing
end

find_drop! (generic function with 1 method)

In [8]:
function flood!(x, y, grid; debug = false, level = 0)
    debug && println(level, ": starting flood! from (", x, ",", y, ")")
    bottom = reach_bottom!(x, y, grid)
    bottom == nothing && return
    x, y = bottom
    debug && println(level, ": reached bottom of (", x, ",", y, ")")
    local lc, rc
    while true
        lc, rc, leak = bounded(x, y, grid)
        if lc != nothing && rc != nothing && !leak
            debug && println(level, ": no leak, filling lc=", lc, ", rc=", rc)
            grid[y, lc.x+1:rc.x-1] .= :water
            debug && println(level, ": water ", lc.x+1, " to ", rc.x-1, " at y=", y)
            y -= 1
        else
            break
        end
    end
    
    # flood right
    debug && println(level, " checking flood right x=", x, " y=", y)
    xp = x + 1
    while xp <= size(grid,2)
        if grid[y, xp] == :clay   # reached right edge
            break
        end
        grid[y, xp] = :water
        if grid[y+1, xp] == :sand
            debug && println(level, " recursion xp=", xp, " y=", y)
            flood!(xp, y, grid, debug = debug, level = level+1)
            break
        end
        xp += 1
    end
    
    # flood left
    debug && println(level, " checking flood left x=", x, " y=", y)
    xp = x - 1
    while xp >= 1
        if grid[y, xp] == :clay   # reached left edge
            break
        end
        grid[y, xp] = :water
        if grid[y+1, xp] == :sand
            debug && println(level, " recursion xp=", xp, " y=", y)
            flood!(xp, y, grid, debug = debug, level = level+1)
            break
        end
        xp -= 1
    end
    
    grid
end

flood! (generic function with 1 method)

In [9]:
points = parse_file("input17_sample.txt")
grid = create_grid(points)
flood!(500, 1, grid; debug = true)
@show count(grid .== :water)
visualize(grid, left = 490)

0: starting flood! from (500,1)
0: reached bottom of (500,6)
0: no leak, filling lc=(x = 495, y = 6), rc=(x = 501, y = 6)
0: water 496 to 500 at y=6
0: no leak, filling lc=(x = 495, y = 5), rc=(x = 501, y = 5)
0: water 496 to 500 at y=5
0: no leak, filling lc=(x = 498, y = 4), rc=(x = 501, y = 4)
0: water 499 to 500 at y=4
0: no leak, filling lc=(x = 498, y = 3), rc=(x = 501, y = 3)
0: water 499 to 500 at y=3
0 checking flood right x=500 y=2
0 recursion xp=502 y=2
1: starting flood! from (502,2)
1: reached bottom of (502,12)
1: no leak, filling lc=(x = 498, y = 12), rc=(x = 504, y = 12)
1: water 499 to 503 at y=12
1: no leak, filling lc=(x = 498, y = 11), rc=(x = 504, y = 11)
1: water 499 to 503 at y=11
1: no leak, filling lc=(x = 498, y = 10), rc=(x = 504, y = 10)
1: water 499 to 503 at y=10
1 checking flood right x=502 y=9
1 recursion xp=505 y=9
2: starting flood! from (505,9)
1 checking flood left x=502 y=9
1 recursion xp=497 y=9
2: starting flood! from (497,9)
0 checking flood left

In [10]:
# another test
points = parse_file("input17_sample.txt")
grid = create_grid(points)
flood!(497, 1, grid)
@show count(grid .== :water)
visualize(grid, left = 490)

count(grid .== :water) = 80
....~~~~~~......#
....~#~~#~~~~...#
....~#~~#~~#~....
....~#~~#~~#~....
....~#~~~~~#~....
....~#~~~~~#~....
....~#######~....
....~.......~....
....~..~~~~~~~~~.
....~..~#~~~~~#~.
....~..~#~~~~~#~.
....~..~#~~~~~#~.
....~..~#######~.


In [11]:
points = parse_file("input17.txt")
grid = create_grid(points)
flood!(500, 1, grid)
@show count(grid .== :water)
# visualize(grid, left = 490)
size(grid)

count(grid .== :water) = 211196


(1984, 697)